In [30]:
!conda install -c conda-forge pandas -y
!conda install -c conda-forge matplotlib -y

import pandas as pd
import re
import matplotlib.pyplot as plt
import numpy as np
import itertools as IT
import glob
import os

Retrieving notices: ...working... done
Channels:
 - conda-forge
 - defaults
Platform: linux-64
Solving environment: done

## Package Plan ##

  environment location: /home/mapa/miniconda3/envs/jupyter

  added / updated specs:
    - pandas


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2024.2.2   |       hbcca054_0         152 KB  conda-forge
    certifi-2024.2.2           |     pyhd8ed1ab_0         157 KB  conda-forge
    numpy-1.26.4               |  py312heda63a1_0         7.1 MB  conda-forge
    openssl-3.2.1              |       hd590300_1         2.7 MB  conda-forge
    pandas-2.2.2               |  py312hfb8ada1_0        14.7 MB  conda-forge
    ------------------------------------------------------------
                                           Total:        24.9 MB

The following packages will be UPDATED:

  ca-certificates                     2023.11.17-hbc

Upload do df

In [26]:
path='/home/mapa/Auto_user/output_plantas_planits'
name = "vsearch"
file='vsearch-final_output.csv'
file_final = path.split('/')[-1]

In [27]:
!echo $path
!echo $file
!echo $name
!echo $file_final

/home/mapa/Auto_user/output_plantas_planits
vsearch-final_output.csv
vsearch
output_plantas_planits


In [28]:
df = pd.read_csv(f'{path}/{file}', sep = '\t', 
                 skiprows=1, header = 0)
df_filter1 = df.query("taxonomy != 'Unassigned'")
df_filter1 = df.loc[df['taxonomy'] != 'Unassigned']
df_filter1




FileNotFoundError: [Errno 2] No such file or directory: '/home/mapa/Auto_user/output_plantas_planits/vsearch-final_output.csv'

In [ ]:
df_filter1_indexed = df_filter1.set_index('taxonomy')
df_filter1_indexed

Transpor o dataframe

In [ ]:
df_transposed = df_filter1_indexed.transpose()
df_transposed

In [ ]:
taxons = list(df_transposed.columns)
count_taxons = pd.melt(df_transposed, value_vars=taxons,value_name='Count', ignore_index=False)

count_taxons = count_taxons.query("Count != 0")

count_taxons = count_taxons.sort_index()
count_taxons['Tool'] = name
count_taxons

In [ ]:
count_taxons.to_csv(f'{path}/count-taxons_{name}.csv',
                                   index=True, sep= '\t')

Substituir contagem acima de 0 pelo nome da coluna

In [ ]:
df_result = df_transposed.where(df_transposed == 0.0, 
                                           df_transposed.columns.to_series(), 
                                           axis=1)
df_result

In [ ]:
# remover colunas e linhas contendo somente zero
df_result_final = df_result.loc[:, (df_result != 0).any(axis=0)]
df_result_final  = df_result_final.loc[(df_result_final !=0).any(axis=1)]
# remove zeros
df_result_final.replace(to_replace = 0.0, value = 'Nan', inplace=True)
# remove NA
# df_result_final.replace(re.compile('NA|,| '), '', inplace=True)
# remove linhas completamente vazias
df_result_final.replace('', np.nan, inplace=True)
df_result_final.dropna(how='all')
df_result_final.stack()
df_result_final

Remover nomes das colunas

In [ ]:
# Define a function to keep unique values in a row
def keep_unique(row):
    unique_values = []
    for item in row:
        if isinstance(item, list):
            unique_values.extend([val for val in item if val not in unique_values])
        elif isinstance(item, str):
            if item not in unique_values:
                unique_values.append(item)
        else:
            if str(item) not in unique_values:
                unique_values.append(str(item))
    return unique_values

# Apply the function using apply and axis=1
df_result_final1 = df_result_final.apply(keep_unique, axis=1)
df_result_final1

In [ ]:
df_result_final2 = pd.DataFrame(df_result_final1)
df_result_final2.columns =['taxonomy']
df_result_final2

Separar cada taxonomia em uma coluna

In [ ]:
df_result_final2.reset_index(inplace=True)
df_result_final2 = df_result_final2.rename(columns = {'index':'sample'})
df_result_final2

In [ ]:
df_split = pd.DataFrame(df_result_final2['taxonomy'].tolist()).fillna('')
df_result_final3 = pd.concat([df_result_final2, df_split], axis=1)
df_result_final3

In [ ]:
df_indexed = df_result_final3.set_index('sample')
df_final = df_indexed.loc[:, df_indexed.columns!='taxonomy']
df_final

In [ ]:
df_final = (df_final.stack()
   .reset_index(level=1, drop=True)
   .reset_index(name='Taxonomy')
)

df_final['Taxonomy'].replace('', np.nan, inplace=True)
df_final['Taxonomy'].replace('Nan', np.nan, inplace=True)
df_final.dropna(subset=['Taxonomy'], inplace=True)
df_final

Salvar como csv

In [ ]:
df_final.to_csv(f'{path}/planilha_{file}',
                                   index=True, sep= '\t')

In [32]:
blast = pd.read_csv(f'{path}/planilha_blast-final_output.csv', sep = '\t', index_col = 'sample')
# blast.rename(columns = {'Taxonomy':'blast'}, inplace = True)
blast['tool'] = 'blast'

vsearch = pd.read_csv(f'{path}/planilha_vsearch-final_output.csv', sep = '\t', index_col = 'sample')
# vsearch.rename(columns = {'Taxonomy':'vsearch'}, inplace = True)
vsearch['tool'] = 'vsearch'

sklearn = pd.read_csv(f'{path}/planilha_sklearn-final_output.csv', sep = '\t', index_col = 'sample')
# sklearn.rename(columns = {'Taxonomy':'sklearn'}, inplace = True)
sklearn['tool'] = 'sklearn'

# Delete column
del sklearn['Unnamed: 0']
del blast['Unnamed: 0']
del vsearch['Unnamed: 0']

frames = [blast, vsearch, sklearn]

result = pd.concat(frames)
#import pandas as pd
# import re
# import matplotlib.pyplot as plt
# import numpy as np
# import itertools as IT
# import glob
# import os
# result = blast.join(sklearn).join(vsearch)
np.unique(result[['Taxonomy', 'tool']].values)
result = result.sort_index()
result


FileNotFoundError: [Errno 2] No such file or directory: '/home/mapa/Auto_user/output_plantas_planits/planilha_blast-final_output.csv'

In [64]:
result.to_csv(f'{path}/final-{file_final}.csv',
                                   index=True, sep= '\t')

In [65]:
files = glob.glob(f'{path}/count-taxons*')
dfs = [pd.read_csv(f, header=0, sep="\t") for f in files]

total_counts = pd.concat(dfs,ignore_index=False)
total_counts = total_counts.groupby(['Unnamed: 0', 'taxonomy', 'Tool']).sum('Count')

In [66]:
total_counts.to_csv(f'{path}/total_counts_{file_final}.csv',
                                   index=True, sep= '\t')